In [ ]:
import arxiv
import requests
import logging
base_url = "https://arxiv.paperswithcode.com/api/v0/papers/"
github_url = "https://api.github.com/search/repositories"
arxiv_url = "http://arxiv.org/"

def get_daily_papers(topic, query="agent", max_results=2):
    """
    @param topic: str
    @param query: str
    @return paper_with_code: dict
    """
    # output
    content = dict()
    content_to_web = dict()
    print("-----------------")
    print(f"query is {query}")
    print("-----------------")
    search_engine = arxiv.Search(
        query=query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate
    )

    for result in search_engine.results():

        paper_id = result.get_short_id()
        paper_title = result.title
        paper_url = result.entry_id
        code_url = base_url + paper_id  # TODO

        paper_abstract = result.summary.replace("\n", " ")

        paper_abstract = paper_abstract.replace("|", ",")
        paper_abstract = paper_abstract.replace("\n", " ")


        primary_category = result.primary_category
        publish_time = result.published.date()
        update_time = result.updated.date()
        comments = result.comment

        # eg: 2108.09112v1 -> 2108.09112
        ver_pos = paper_id.find('v')
        if ver_pos == -1:
            paper_key = paper_id
        else:
            paper_key = paper_id[0:ver_pos]
        paper_url = arxiv_url + 'abs/' + paper_key

        try:
            # source code link
            r = requests.get(code_url).json()
            repo_url = None
            if "official" in r and r["official"]:
                repo_url = r["official"]["url"]

            if repo_url is not None:
                content[paper_key] = "|**{}**|**{}**|[{}]({})|**[link]({})**|**{}**|\n".format(
                    update_time, [paper_title](paper_url), paper_key, paper_url, repo_url, paper_abstract)
                content_to_web[paper_key] = "- {}, **{}**, Paper: [{}]({}), Code: **[{}]({})**".format(
                    update_time, [paper_title](paper_url), paper_url, paper_url, repo_url, repo_url)

            else:
                content[paper_key] = "|**{}**|**{}**|[{}]({})|null|{}|\n".format(
                    update_time, [paper_title](paper_url), paper_key, paper_url, paper_abstract)
                content_to_web[paper_key] = "- {}, **{}**, Paper: [{}]({}),{}".format(
                    update_time, [paper_title](paper_url), paper_url, paper_url, paper_abstract)

            # TODO: select useful comments
            comments = None
            if comments != None:
                content_to_web[paper_key] += f", {comments}\n"
            else:
                content_to_web[paper_key] += f"\n"

        except Exception as e:
            logging.error(f"exception: {e} with id: {paper_key}")

    data = {topic: content}
    data_web = {topic: content_to_web}
    return data, data_web



In [3]:
import arxiv
import requests
import logging
base_url = "https://arxiv.paperswithcode.com/api/v0/papers/"
github_url = "https://api.github.com/search/repositories"
arxiv_url = "http://arxiv.org/"
query = "computational argumentation"
max_results = 100
 

search_engine = arxiv.Search(
    query=query,
    max_results=max_results,
    sort_by=arxiv.SortCriterion.SubmittedDate
)

for result in search_engine.results():

    paper_id = result.get_short_id()
    paper_title = result.title
    paper_comment = result.comment
    print(paper_comment)
    paper_url = result.entry_id
    code_url = base_url + paper_id  # TODO

C:\Users\Administrator\AppData\Local\Temp\ipykernel_63512\441346572.py:17: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search_engine.results():


Technical Report v1 (44 pages, 23 figures, project page:
  https://sgp-bench.github.io/)
None
32 pages
Pre-print. 11 pages main, 8 pages app., 28 figures
None
None
None
This paper is an extension of our ICCV 23 paper (arXiv:2303.05118)
None
16 pages, 8 figures, 2 tables
None
None
33 pages, 14 figures
None
None
10 pages, 6 figures, 5 tables
None
None
None
None
None
for companion code, see https://github.com/bob-carpenter/adaptive-hmc
Accepted for ECCV 2024
34 pages. arXiv admin note: text overlap with arXiv:2402.19271
None
None
20 pages, 10 figures, 2 tables, submitted to PRC
13 pages, 11 figures
None
None
None
None
None
249 plots in 48 figures, and 81 tables. This is an extended version
  of the paper Linares L\'opez, Carlos and Herman, Ian. 2024. Evolving A* to
  Efficiently Solve the k Shortest-Path Problem. Proceedings of the European
  Conference on Artificial Intelligence (ECAI). To appear
Preprint accepted for 15th International Conference on Computational
  Logistics
Accepted by

In [10]:
import re

# 定义正则表达式
pattern = r"(?i)(?<=accepted by\s).*"

# 定义一些示例文本
texts = [
    "Accepted by ICANN 2024",
    "8 pages, 4 figures, accepted by ECAI",
    "10 pages, 3 figures",
    "accepted by ACM MM 2024"
]

# 遍历所有文本，使用正则表达式匹配
for text in texts:
    match = re.search(pattern, text)
    if match:
        print(f"In '{text}', found: '{match.group()}'")
    else:
        print(f"In '{text}', no match found.")

In 'Accepted by ICANN 2024', found: 'ICANN 2024'
In '8 pages, 4 figures, accepted by ECAI', found: 'ECAI'
In '10 pages, 3 figures', no match found.
In 'accepted by ACM MM 2024', found: 'ACM MM 2024'


In [11]:
print(match.group())

ACM MM 2024


In [20]:
import re

# 定义正则表达式
pattern = r"(?i)\b(AAAI|NeurIPS|ACL|CVPR|ICCV|ICML|IJCAI|EMNLP|COLING|NAACL|EACL|CoNLL|ICLR|INLG|ECCV|MICCAI|IROS|MIDL|ICIP|KDD|MIDC)(?:\s?(\d{4}))?\b"

# 定义一些示例文本
texts = [
    "Accepted by  2024 AAAI",
    "Submitted to NeurIPS",
    "This is a CVPR2023 paper.",
    "Interesting results in ICLR",
    "MICCAI 2022 had great papers",
    "NAACL2021 was a virtual event",

]

# 遍历所有文本，使用正则表达式匹配
for text in texts:
    matches = re.findall(pattern, text)
    if matches:
        # 格式化输出匹配结果
        formatted_matches = [f"{conf} {year}".strip() for conf, year in matches]
        print(f"In '{text}', found conferences: {', '.join(formatted_matches)}")
    else:
        print(f"In '{text}', no conference name found.")

In 'Accepted by  2024 AAAI', found conferences: AAAI
In 'Submitted to NeurIPS', found conferences: NeurIPS
In 'This is a CVPR2023 paper.', found conferences: CVPR 2023
In 'Interesting results in ICLR', found conferences: ICLR
In 'MICCAI 2022 had great papers', found conferences: MICCAI 2022
In 'NAACL2021 was a virtual event', found conferences: NAACL 2021


In [23]:
print(formatted_matches)
print(', '.join(formatted_matches))
a= ', '.join(formatted_matches)
print(a)

['NAACL 2021']
NAACL 2021
NAACL 2021


In [6]:
import arxiv
import requests
import logging
base_url = "https://arxiv.paperswithcode.com/api/v0/papers/"
github_url = "https://api.github.com/search/repositories"
arxiv_url = "http://arxiv.org/"
query = "debate"
max_results = 100
 

search_engine = arxiv.Search(
    query=f'ti:{query}',
    max_results=max_results,
    sort_by=arxiv.SortCriterion.SubmittedDate,
    
)

for result in search_engine.results():

    paper_id = result.get_short_id()
    paper_title = result.title
    paper_comment = result.comment
    print(paper_title)
    paper_url = result.entry_id
    code_url = base_url + paper_id  # TODO

C:\Users\Administrator\AppData\Local\Temp\ipykernel_17880\1422836124.py:18: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search_engine.results():


Defense Priorities in the Open-Source AI Debate: A Preliminary Assessment
Can LLMs Beat Humans in Debating? A Dynamic Multi-agent Framework for Competitive Debate
DebateQA: Evaluating Question Answering on Debatable Knowledge
Interpreting and Mitigating Hallucination in MLLMs through Multi-agent Debate
Overview of AI-Debater 2023: The Challenges of Argument Generation Tasks
MentalAgora: A Gateway to Advanced Personalized Care in Mental Health through Multi-Agent Debating and Attribute Control
Unlocking Varied Perspectives: A Persona-Based Multi-Agent Framework with Debate-Driven Text Planning for Argument Generation
Sequence Graph Network for Online Debate Analysis
MultiAgent Collaboration Attack: Investigating Adversarial Attacks in Large Language Model Collaborations via Debate
Mining United Nations General Assembly Debates
Debate as Optimization: Adaptive Conformal Prediction and Diverse Retrieval for Event Extraction
Improving Multi-Agent Debate with Sparse Communication Topology
C